In [14]:
import openai
import os

In [13]:
print(openai.__version__)

0.27.8


In [8]:
openai.api_key = os.getenv("OPENAI_API_KEY")
client = openai.OpenAI()

AttributeError: module 'openai' has no attribute 'OpenAI'

In [6]:
assistant = client.beta.assistants.create(
  instructions="You are a customer support chatbot. Use your knowledge base to best respond to customer queries.",
  model="gpt-4-1106-preview",
  tools=[{"type": "retrieval"}]
)